In [2]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
import imutils
import scipy.stats as stats

In [3]:
## Ne garder que les images non câblées

liste_non_cablees=[]
dossier="ModulePictures"

for fichier in os.listdir(dossier):
    if os.path.isfile(os.path.join(dossier, fichier)) and ("Reception" in os.path.basename(fichier)):
        liste_non_cablees.append(str(dossier + "/"+ os.path.basename(fichier)))

## Ne garder que les images câblées

liste_cablees=[]
dossier="ModulePictures"

for fichier in os.listdir(dossier):
    if os.path.isfile(os.path.join(dossier, fichier)) and ("AfterBonding" in os.path.basename(fichier)):
        liste_cablees.append(str(dossier + "/"+ os.path.basename(fichier)))


In [16]:
def mires(img_input, draw = False):
    
    assert img_input is not None, "file could not be read, check with os.path.exists()"

    img = cv.cvtColor(img_input,cv.COLOR_BGR2GRAY)

    (height,length) = img.shape

    ## TODO : changer cette extraction à la main par quelque chose d'automatisé
    sliceparams = [(100, 400, 300, 600, 1),  
                   (-400, -100, 300, 600, 1),
                   (3800, 4200, 500, 800, 2), 
                   (100, 400, -600,-300, 1), 
                   (-400, -100, -600, -300, 1), 
                   (3800, 4200, -800, -500, 2)]

    cimg = cv.cvtColor(img,cv.COLOR_GRAY2BGR)

    centers = []

    for (beg1, end1, beg2, end2, nbmires) in sliceparams :
        mask = img[beg1:end1, beg2:end2]

        #Blur pour réduire le bruit
        mask = cv.medianBlur(mask,5)
        #Fonction qui détecte les cercles.
        circles = cv.HoughCircles(mask,cv.HOUGH_GRADIENT,1,minDist = 100,
                                    param1=50,param2=20,minRadius=20 ,maxRadius=30)

        if circles is not None:
            circles = np.int16(np.around(circles))
            for i in circles[0,:nbmires]:
                
                centers.append((i[1]+beg1%height, i[0]+beg2%length)) #Attention ici j'inverse x,y afin de travailler en lignes/colonnes

                #Dessin des cercles
                if draw :
                    cv.circle(cimg,(i[0]+beg2%length,i[1]+beg1%height),i[2],(0,255,0),20)
                    cv.circle(cimg,(i[0]+beg2%length,i[1]+beg1%height),2,(0,0,255),3)

    if draw :
        plt.imshow(cimg)
        plt.show()

    assert len(centers) == 8, "Mires manquantes"

    slope = sum([(centers[i+4][0]-centers[i][0])/(centers[i+4][1]-centers[i][1]) for i in range(0,4)])/4

    return centers,slope

In [ ]:
#Détection à la main de la frontière supérieure verte :

def slope_pcb(img) :

    lower_bound = np.array([0, 40, 0])
    upper_bound = np.array([40,110,110])
    bwimg = cv.inRange(img, lower_bound, upper_bound)
    imagemask_green = cv.medianBlur(bwimg,25)

    middle = img.shape[1]//2

    top_contour = np.zeros((1000,2),dtype=np.int32)
    bot_contour = np.zeros((1000,2),dtype=np.int32)

    for j in range(middle - 2000, middle - 1000, 2) :

        top_contour[(j - middle + 2000)//2] = [j,min(np.where(imagemask_green[:,j] == 255)[0])]
        bot_contour[(j - middle + 2000)//2] = [j,max(np.where(imagemask_green[:,j] == 255)[0])]

    for j in range(middle + 1000, middle + 2000, 2) : 

        top_contour[(j - middle)//2] = [j,min(np.where(imagemask_green[:,j] == 255)[0])]
        bot_contour[(j - middle)//2] = [j,max(np.where(imagemask_green[:,j] == 255)[0])]

    slope = 0.5*(stats.linregress(top_contour[:,0], top_contour[:,1]).slope + stats.linregress(bot_contour[:,0], bot_contour[:,1]).slope)

    return slope

In [18]:
## Fonctions utiles

def tourner_image(image, pente):
    return imutils.rotate(image, np.arctan(pente))

def trouver_la_paire(fichier, dossier):
    bname=os.path.basename(fichier)
    if "AfterBonding" in bname:
        name=bname[:bname.find("AfterBonding")]
        for f in os.listdir(dossier):
            if "Reception" in os.path.basename(f):
                if os.path.basename(f)[:os.path.basename(f).find("Reception")]==name:
                    return f
    elif "Reception" in bname:
        name=bname[:bname.find("Reception")]
        for f in os.listdir(dossier):
            if "AfterBonding" in os.path.basename(f):
                if os.path.basename(f)[:os.path.basename(f).find("AfterBonding")]==name:
                    return f
    return "Pas de paire"

In [19]:
trouver_la_paire(liste_cablees[0],"ModulePictures")

'20UPGM23210250_PPPV2_45_Reception_Glo_NoLight_AfterClean.jpg'

In [40]:
def afficher(img) :
    cv.imshow("Image", img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
def rotation_mires(path): #prend en argument une image cablée, renvoie la rotation des mires.
    img_cablee = cv.imread(path)
    path_non_cablee = 'ModulePictures/' + trouver_la_paire(path,"ModulePictures")
    img_non_cablee = cv.imread(path_non_cablee)

    relative_slope = slope_pcb(img_non_cablee) - mires(img_non_cablee)[1]

    return tourner_image(img_cablee, slope_pcb(img_cablee) - relative_slope)

In [64]:
afficher(rotation_mires(liste_cablees[0]))

[[[11  7  2]
  [ 5  1  0]
  [14 10  5]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 4  1  0]
  [ 4  1  0]
  [ 5  2  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 1  1  1]
  [ 5  5  5]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 ...

 [[ 4  6  7]
  [ 1  3  4]
  [ 5  4  8]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  1]
  [ 0  0  1]
  [ 5  1  6]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  2  3]
  [ 0  0  1]
  [ 4  0  5]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]]


In [24]:
img_cablee = cv.imread(liste_cablees[0])
slope_pcb(img_cablee)


np.float64(-0.0003850708407244058)

In [51]:
'ModulePictures/' + trouver_la_paire(liste_cablees[0],"ModulePictures")

'ModulePictures/20UPGM23210250_PPPV2_45_Reception_Glo_NoLight_AfterClean.jpg'

In [57]:
liste_non_cablees[0]

'ModulePictures/20UPGM23210250_PPPV2_45_Reception_Glo_NoLight_AfterClean.jpg'

In [60]:
cv.imread(liste_non_cablees[0])

array([[[11,  7,  2],
        [ 5,  1,  0],
        [14, 10,  5],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 4,  1,  0],
        [ 4,  1,  0],
        [ 5,  2,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 1,  1,  1],
        [ 5,  5,  5],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       ...,

       [[ 4,  6,  7],
        [ 1,  3,  4],
        [ 5,  4,  8],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  1],
        [ 0,  0,  1],
        [ 5,  1,  6],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  2,  3],
        [ 0,  0,  1],
        [ 4,  0,  5],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]]], dtype=uint8)